# Training Pytorch CNN Model to work on MNIST Dateset

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elyas1376/ComputerVision/blob/main/CNN/CNN_keras.ipynb)

In [35]:
#@title Imports 

!pip install torchviz
 
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plot
import torch
from torch import nn
from torch.utils.data import DataLoader , Dataset
from torchvision.datasets import MNIST
from torchvision import transforms
import matplotlib.pyplot as plt
from time import time
from torchviz import make_dot

clear_output()

In [36]:
#@title Loading MNIST Data


#Creating a Transformer to Augment and Implement required setting on data
My_Transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5)
])

#=========== Fetching MNIST Data ===============
train_data  = MNIST(root="./data",download=True,train = True,transform=My_Transformer)
test_data   = MNIST(root="./data",download=False,train= False,transform=My_Transformer)
#==============================================


#============ Creating Dataloaders ============
train_loader= DataLoader(dataset = train_data,batch_size = 64,num_workers=2)
test_loader = DataLoader(dataset = test_data ,batch_size = 64,num_workers=2)
#==============================================

In [37]:
#@title MNIST NN Architecture
class MNIST_classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(1,16,kernel_size=3)
    self.relu1=nn.ReLU()
    self.conv2=nn.Conv2d(16,16,kernel_size=3)
    self.relu2=nn.ReLU()
    self.pool1=nn.MaxPool2d(kernel_size=2, stride=2 )
    self.dropout1=nn.Dropout2d(0.2)
    self.flatten=nn.Flatten()
    #FC
    self.fc1=nn.Linear(in_features=2304, out_features=10)

  def forward(self,input):
    y= self.conv1(input)
    y= self.conv2(y)
    y= self.pool1(y)
    y= self.dropout1(y)
    y= self.flatten(y)
    y= self.fc1(y)
    return y

    
def print_done(text ,upper=False,lower=False):
  
  if upper:
    print("*"*60)

  print(text)
  
  if lower:
    print("*"*60)



#Creating the model
device = ('cuda' if torch.cuda.is_available() else 'cpu') #Use Gpu if availbe
Mnist_model = MNIST_classifier().to(device)

In [ ]:
#@title Model Summary and Ploting the Arch
print("Model Summary:\n",Mnist_model)

#================Ploting the Model==================
batch = next(iter(train_loader))
yhat  = Mnist_model(batch[0])

print_done("Model Graph",upper=True,lower=True)
make_dot(yhat, params=dict(list(Mnist_model.named_parameters())))




In [ ]:
#@title Train loop

#=======================Train Loop======================
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = Mnist_model.parameters())

losses = []
val_losses= []
train_accuracy = []
val_accuracy   = []

for epoch in range(epochs):
  
  START = time()
  
  #============ Training ===============
  Mnist_model = Mnist_model.train()
  batch_losses = []
  epoch_loss = []
  epoch_corrects = 0

  for batch in train_loader:
    input = batch[0].to(device) 
    target= batch[1].to(device) 
    output = Mnist_model.forward(input)

    #loss
    optimizer.zero_grad()
    loss=criterion(output,target)
    loss.backward()
    optimizer.step()
    batch_losses.append(loss.item())
    batch_corrects = sum(target == output.argmax(dim=1))
    epoch_corrects += batch_corrects
  
  epoch_loss = np.mean(batch_losses)
  losses.append(epoch_loss)

  epoch_accuracy = epoch_corrects/len(train_data)
  train_accuracy.append(epoch_accuracy)
  train_str = f"Epoch: {epoch} ,loss: {epoch_loss:0.5f} ,accuracy: {epoch_accuracy:0.5f}"

  Mnist_model = Mnist_model.eval()

  #============ Validation ===============
  batch_losses = []
  epoch_corrects = 0

  for batch in test_loader:
    input = batch[0].to(device) 
    target= batch[1].to(device) 
    output = Mnist_model.forward(input)

    #loss
    optimizer.zero_grad()
    loss=criterion(output,target)
    loss.backward()
    optimizer.step()
    batch_losses.append(loss.item())
    batch_corrects = sum(target == output.argmax(dim=1))
    epoch_corrects += batch_corrects
  
  epoch_loss = np.mean(batch_losses)
  val_losses.append(epoch_loss)

  epoch_accuracy = epoch_corrects/len(test_data)
  val_accuracy.append(epoch_accuracy)



  END= time()
  test_str=f" , val_loss: {epoch_loss: 0.5f} ,val_accurancy : {epoch_accuracy: 0.5f} , time: {END -START: 0.2f}"


  LOOP_str  = train_str +  test_str


  print(LOOP_str)

  #Early Stopping for by measure of loss O:)
  if epoch_loss <= 0.001:
    print_done(f"Optimal loss ({loss:0.4f}) and val_loss :({epoch_accuracy:0.4f}) is reached Congrats!",upper=True)
    print_done("Model parameters",*[f"{param.item():.1f}" for param in model.parameters()],lower=True)
    break


print("Training Finished")


In [ ]:
#@title Loss visualization

fig,ax = plt.subplots(nrows=1, ncols=2 , figsize=(18,5))
ax[0].set_title(f"Highest accuracy %{train_accuracy[-1]:0.2f}")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Loss")
ax[0].plot(losses,'blue',lw=3,ls='--');


ax[1].set_title(f"Highest validation accuracy %{val_accuracy[-1]:0.2f}")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Val Loss")
ax[1].plot(val_losses,'green',lw=3,ls='dotted');